In [2]:
import torch
from transformers import BitsAndBytesConfig

quantization_config  = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True
    )

model_id = 'mistralai/Mistral-7B-Instruct-v0.1'

In [3]:
## carregando o modelo instruct todo, com configurações de quantização

from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
model_4bit = AutoModelForCausalLM.from_pretrained(model_id,device_map="auto",quantization_config=quantization_config)
tokenizer = AutoTokenizer.from_pretrained(model_id)

/home/gean/anaconda3/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Could not find the bitsandbytes CUDA binary at PosixPath('/home/gean/anaconda3/lib/python3.11/site-packages/bitsandbytes/libbitsandbytes_cuda118.so')
The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

AttributeError: /home/gean/anaconda3/lib/python3.11/site-packages/bitsandbytes/libbitsandbytes_cpu.so: undefined symbol: cquantize_blockwise_fp16_nf4

In [ ]:

pipeline = pipeline('text-generation', 
                    model=model_4bit, 
                    tokenizer=tokenizer,
                    use_cache=True,
                    device_map='auto',
                    max_length=1000,
                    do_sample=True,
                     top_k=5,
                     num_return_sequences=1,
                     eos_token_id=tokenizer.eos_token_id,
                     pad_token_id=tokenizer.pad_token_id)

from langchain import HuggingFacePipeline
from langchain import PromptTemplate, LLMChain

llm = HuggingFacePipeline(pipeline=pipeline)


In [1]:
from ctransformers import AutoModelForCausalLM
# Set gpu_layers to the number of layers to offload to GPU. Set to 0 if no GPU acceleration is available on your system.
llm = AutoModelForCausalLM.from_pretrained("TheBloke/Mixtral-8x7B-Instruct-v0.1-GGUF", model_file="/home/gean/Documentos/mixtral-8x7b-instruct-v0.1.Q4_K_M.gguf", model_type="mistral", gpu_layers=50)


Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

config.json:   0%|          | 0.00/31.0 [00:00<?, ?B/s]

Fetching 0 files: 0it [00:00, ?it/s]

RuntimeError: Failed to create LLM 'mistral' from '/home/gean/Documentos/mixtral-8x7b-instruct-v0.1.Q4_K_M.gguf'.

In [5]:
prompt  ='<s>[INST] you are a helpul assistant. your role is to explain what is like to be a physicist and data scientist.your answer must be in Brazilian Portuguese.[/INST]'

In [6]:
print(llm(prompt))

 Alguma diferença existe entre ser um físico e um cientista de dados em brasileiro? Sim, existem certas distinções, como mencionei. Os fisicos são especialistas em investigar o universo natural, incluindo a matemática que descreve as leis da natureza. Por exemplo, os fisicos desejam entender como o universo funciona, como o electrão se move, como as estrelas se formam e muito mais.

Em contrário, os cientistas de dados são especialistas em analisar e interpretar dados coletados a partir de fontes variadas. É como quando precisamos saber quanto tempo cada mes passou de caro para comprar um carro no Brasil. Nesse caso, temos dados relacionados com o preço do carro, a quantidade de dólares nele presente e outras informações relevantes. Então, os cientistas de dados são os responsáveis por tentar interpretar estas informações e entender como o mercado funciona no Brasil.


In [17]:
llm_chain = LLMChain(prompt=prompt,llm=llm)
response = llm_chain.run({"question":question_p,"context":context_p})

/home/gean/anaconda3/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


TypeError: unhashable type: 'list'

In [20]:
from langchain.llms import CTransformers
config = {'max_new_tokens': 1000, 'temperature': 0.7,'max_new_tokens':2060,'context_length':4098}
llm = CTransformers(model="TheBloke/Mistral-7B-Instruct-v0.1-GGUF",
                    model_file="/home/gean/Documentos/mistral-7b-instruct-v0.1.Q4_K_M.gguf",
                    config=config)


Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Fetching 0 files: 0it [00:00, ?it/s]

In [25]:

%%time 
template = """<s>[INST] You are a helpful, respectful and honest assistant. Answer  exactly in few words from the context.
Write your answer in portuguese.
Do your task from the context below: 
{context}
{task} [/INST]</s>
"""

question_p = """Resuma sobre o que foi a conversa"""
context_p = """Falante 0 (Atendente): Olá, bom dia! Aqui é o Carlos da Rede Brasil, prestando serviço para o Banco Itaú. Por gentileza, poderia confirmar seu nome?

Falante 1 (Cliente): Oi, bom dia. Sou o Roberto, sim.

Falante 0: Obrigado, Roberto. Verificando aqui... o senhor possui uma pendência conosco e queríamos oferecer algumas opções para resolver isso. Tem um momento para discutirmos?

Falante 1: Ah, claro, pode falar.

Falante 0: Excelente! Temos uma oportunidade para pagamento à vista com um desconto de 40%, o que reduziria bastante o valor total. Caso prefira, também podemos arranjar um plano de pagamento parcelado com um desconto menor. Como lhe parece?

Falante 1: Hum, e quanto seria o desconto para o parcelado?

Falante 0: Para o parcelamento, podemos fazer com 20% de desconto, e o valor poderia ser dividido em até 12 vezes sem juros. Isso ajudaria a ajustar melhor no seu orçamento.

Falante 1: E qual seria o valor das parcelas então?

Falante 0: Um momento, vou calcular aqui... [barulho de teclas] Certo, as parcelas ficariam em torno de duzentos reais cada uma.

Falante 1: Tá, parece interessante. Vou precisar pensar um pouco, posso retornar a ligação mais tarde?

Falante 0: Claro, sem problemas! Mas gostaria de lembrar que essas condições são por tempo limitado. Posso reservar a oferta até o final do dia de hoje. Como deseja proceder?

Falante 1: Okay, eu ligo até o final do dia então. Obrigado, Carlos.

Falante 0: Por nada, Roberto. Estamos à disposição para qualquer dúvida. Tenha um ótimo dia!

Falante 1: Igualmente, até mais.
"""

prompt = PromptTemplate(template=template, input_variables=["task","context"])
llm_chain = LLMChain(prompt=prompt,llm=llm)
response = llm_chain.invoke({"task":question_p,"context":context_p})

CPU times: user 7min 13s, sys: 449 ms, total: 7min 13s
Wall time: 54.9 s


In [26]:
response.keys()

dict_keys(['task', 'context', 'text'])

In [28]:
print(response['text'])


A conversa entre Falante 0 (Atendente) e Falante 1 (Cliente) foi sobre uma pendência que o cliente possui conosco e queremos oferecer algumas opções para resolver isso. O atendente explica duas opções: pagamento à vista com um desconto de 40% e plano de pagamento parcelado com um desconto menor, com valor dividido em até 12 vezes sem juros. Falante 1 precisa pensar mais uma vez antes de decidir, mas pode retornar a ligação até o final do dia do hoje e que estas condições são temporárias. O cliente obrigado e atendente disponível para qualquer dúvida.


In [29]:
import pandas as pd

In [38]:
data = pd.read_csv("/home/gean/Documentos/Conversas_Ruidosas_Dataset.csv")

In [42]:
from langchain.llms import CTransformers
config = {'max_new_tokens': 1000, 'temperature': 0.7,'max_new_tokens':2060,'context_length':4098}
llm = CTransformers(model="TheBloke/Mistral-7B-Instruct-v0.1-GGUF",
                    model_file="/home/gean/Documentos/mistral-7b-instruct-v0.1.Q4_K_M.gguf",
                    config=config)

template = """<s>[INST] You are a helpful, respectful and honest assistant. Answer  exactly in few words from the context.
Write your answer in portuguese.
Do your task from the context below: 
{context}
{task} [/INST]</s>
"""

def generate_response(tempĺate,contexto,task="""Resuma sobre o que foi a conversa"""):
    prompt = PromptTemplate(template=template, input_variables=["task","context"])
    llm_chain = LLMChain(prompt=prompt,llm=llm)
    response = llm_chain.invoke({"task":task,"context":contexto})
    return response

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Fetching 0 files: 0it [00:00, ?it/s]

In [44]:
resposta = generate_response(template,data.texto.values[45])

In [53]:
%%time
output_data = pd.DataFrame()
for i in range(data.shape[0]):
    resposta = generate_response(template,data.texto.values[i])
    output_data = pd.concat([output_data,pd.DataFrame({"id":[i],"resumo":[resposta['text']]})])
    break

CPU times: user 4min 13s, sys: 356 ms, total: 4min 13s
Wall time: 32.1 s


In [55]:
output_data.resumo.values[0]

'\nA conversa se referiu ao fardo financeiro de um cliente com Banco Itaú e discutiu opções para resolver a situação, como pagamento à vista com desconto ou parcelamento. O desconto à vista seria até 40%, e o parcelamento teria um desconto de 20% com as parcelas divididas em até 12 meses. A conversa foi interrompida e seria continuada por email.'

In [59]:
data.texto.values[0]

'Falante erm 0: Olá, da Rede Brasil sobre sua ,, dívida com Banco Itaú. Tem um minuto? uh Falante 1: Sim, erm estou disponível agora, pode falar. Falante 0: Temos registros de uma pendência financeira em seu nome. Gostaria de discutir algumas opções para resolver isso. Falante 1: Quais são as opções? Falante 0: Podemos oferecer um desconto para pagamento à vista ou opções de parcelamento ajustáveis ao seu orçamento. Falante 1: E o desconto à vista seria de quanto? Falante 0: Até 40% de desconto sobre o total devido. É hmm uma excelente oportunidade. Falante 1: E o parcelamento, como funciona? Falante 0: O desconto erm seria oh de erm 20% e as parcelas poderiam ser divididas em até 12 meses. Falante 1: Vou precisar de mais informações antes de decidir. Você pode enviar por email? Falante 0: Claro, enviarei oh todos os detalhes hmm e podemos marcar outra conversa para discutir uh mais. Falante 1: uh Ótimo, aguarde seu contato. Obrigado. '

In [58]:
data

,id_conversa,texto
0,1,"Falante erm 0: Olá, da Rede Brasil sobre sua ,..."
1,2,"Falante 0: Olá, ,, da Rede ah Brasil sobre sua..."
2,3,"Falante 0: Olá, da Rede Brasil ah sobre sua dí..."
3,4,"Falante 0: Olá, oh da Rede ... Brasil sobre su..."
4,5,"Falante 0: Olá, da Rede Brasil erm sobre sua d..."
...,...,...
995,996,"Falante 0: Olá, da Rede Brasil sobre sua dívid..."
996,997,"Falante 0: Olá, da Rede Brasil sobre ,, sua hm..."
997,998,"Falante 0: Olá, da Rede Brasil oh sobre sua dí..."
998,999,"Falante 0: Olá, da Rede Brasil sobre sua dívid..."


## Mixtral

In [ ]:
from llama_cpp import Llama

# Set gpu_layers to the number of layers to offload to GPU. Set to 0 if no GPU acceleration is available on your system.
llm = Llama(
  model_path="/home/gean/Documentos/mixtral-8x7b-instruct-v0.1.Q4_K_M.gguf",  # Download the model file first
  n_ctx=2048,  # The max sequence length to use - note that longer sequence lengths require much more resources
  n_threads=8,            # The number of CPU threads to use, tailor to your system and the resulting performance
  n_gpu_layers=35         # The number of layers to offload to GPU, if you have GPU acceleration available
)
